In [77]:
pip install emoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
pip install emojis|

/bin/bash: -c: line 1: syntax error: unexpected end of file


In [80]:
pip install pyemoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
# installing specific version :https://pypi.org/project/xgboost/1.3.1/
# pip install xgboost==1.3.1


In [82]:
pip install emojis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:

import emojis

import string
from tensorflow.keras.preprocessing.text import Tokenizer 

In [84]:
import pickle
import numpy as np 
import pandas as pd 
import tensorflow as tf
from xgboost import XGBClassifier #1.3.1
import pyemoji
from cleantext import clean
import cv2 as cv
import regex as re


In [85]:
datafile  = pd.read_csv("/content/Post1and2.csv")
datafile2 = pd.read_csv("/content/Post1and2.csv")
filename = "/content/ModelForEmojiClf.sav"
emoji_model = pickle.load(open(filename,'rb'))


In [86]:
def classifier(l,c,e,q): 
    tot_count = e+c
    if tot_count >=100: 
        w = int(l+(e/c)*50)
    else: 
        w = int(l+(e/c)*20)
    if q == "high": 
        return round(w)
    elif q == "medium": 
        return round(w/2)
    else: 
        return 0

def emoji_extractor(sentence): 
    return [sentence[i] for i in range(len(str(sentence))) if str(sentence[i].encode("unicode-escape"))[2]=='\\']

def ImageQuality_assement(img): 
    image = cv.imread(str(img))
    y, x = image.shape[0], image.shape[1]
    pixel = int(y*x)
    if pixel <= 40000: 
        return "poor"
    elif pixel > 40000 and pixel <= 150000: 
        return "medium"
    else: 
        return "high"
def emoji_clf(emoji):
    em = str(emoji)
    temp = pyemoji.entities(em)[2:-1]
    x = len(re.findall('[0-9]+', temp))
    if x >0: 
        arr = np.array(float(temp)).reshape(-1,1)
        predicted = emoji_model.predict(arr)
        return list(predicted)

def extract_emojis(s):
    t = len(emojis.get(s))
    if t != 0 :
        return s
    else: 
        pass
def remove_emoji(text): 
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def remove_url(text):
    url_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)

def clean_text(text):
    delete_dict = {sp_character: '' for sp_character in string.punctuation}
    delete_dict[' '] = ' '
    table = str.maketrans(delete_dict)
    text1 = text.translate(table) 
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))])
    return text2.lower()        
    


test_data = datafile
test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_text'] = test_data['text'].apply(lambda x:len(str(x).split())) 
max_test_sentence_length  = test_data['Num_words_text'].max()

mask = test_data['Num_words_text'] >2
test_data = test_data[mask]



test_data['text'] = test_data['text'].apply(remove_emoji)
test_data['text'] = test_data['text'].apply(remove_url)
test_data['text'] = test_data['text'].apply(clean_text)

num_words = 20000 
tokenizer = Tokenizer(num_words=num_words, oov_token="unk") 
tokenizer.fit_on_texts(test_data['text'].tolist()) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [87]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_test  = np.array( tokenizer.texts_to_sequences(test_data['text'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [88]:
from tensorflow.keras.models import model_from_json
json_file = open('/content/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/model.h5")

In [89]:
x_test 

array([[ 94,  15,  10, ...,   0,   0,   0],
       [ 99,  31,   7, ...,   0,   0,   0],
       [  4,   3,  14, ...,   0,   0,   0],
       ...,
       [  4,   3,  14, ...,   0,   0,   0],
       [ 33, 246,  89, ...,   0,   0,   0],
       [ 79,   5,  77, ...,   0,   0,   0]], dtype=int32)

In [90]:
print("Generate predictions for all samples")
predstate= tf.convert_to_tensor(x_test,dtype=tf.int64)
predictions = loaded_model.predict(predstate)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples
2/2 [==============================] - 0s 5ms/step


In [91]:
test_data['pred_sentiment']= predict_results
test_data['pred_sentiment'] = np.where((test_data.pred_sentiment == 0),'class1',test_data.pred_sentiment)
test_data['pred_sentiment'] = np.where((test_data.pred_sentiment == '1'),'class2',test_data.pred_sentiment)
test_data['pred_sentiment'] = np.where((test_data.pred_sentiment == '2'),'class3',test_data.pred_sentiment)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [92]:
def recommender_system(filedata,image): 
    global emojiS,emofil
    
    txt_data = filedata[['likesCount','text']].dropna()
    l = int(txt_data['likesCount'].sum())
    print("No. of Likes: ",l)
    c = int(len(filedata['text']))
    print("No. of Commontes: ",c)
    emojiS = emoji_extractor("".join(list(filedata['text'])))
    e = int(len(emojiS))
    print("No. of emojis: ",e)
    q = ImageQuality_assement(str(image))
    print("ImageQuality: ",q)
    Wt = classifier(l,c,e,q)
    print("No. of Sales Estimated (Wt): ",Wt) 
    
    temp = list(map(extract_emojis,emojiS))
    
    emofil = [i for i in temp if i is not None]
    
    emoji_classes = np.array(list(map(emoji_clf,emofil)),ndmin=1)
    emoji_classes = list(emoji_classes.flatten())
    
    print("Positive Emoji Count: ",emoji_classes.count([1]))
    print("negative Emoji Count: ",emoji_classes.count([2]))
    print("neutral Emoji Count: ",emoji_classes.count([0]))
    print()
    print() 
    print("Suggestions")
    
    if q == "poor":
        print("Need to improve image quality about by 50%")
    elif q == "medium": 
        print("Need to improve image quality by 30%")
    else: 
        form = round(int(c/3))
        if emoji_classes.count([2]) >= form:
            print("suggested to improve the quality of the product")
        elif emoji_classes.count([1]) >= form: 
            print("Good review overall, stay consistent and provide Quality Product")
        elif emoji_classes.count([0]) >= form:
            print("Improve the quality of the product a bit more")
    
    print("The text Classification Report")
    print()
    print(test_data['pred_sentiment'].value_counts())
    
    m = max(test_data['pred_sentiment'].value_counts())
    print("The Maximum Class output is: ",m)
    

    
    
    
    
    

In [93]:
recommender_system(datafile2,"/content/Post1_2.jpg")

No. of Likes:  38
No. of Commontes:  326
No. of emojis:  46
ImageQuality:  high
No. of Sales Estimated (Wt):  45
Positive Emoji Count:  3
negative Emoji Count:  4
neutral Emoji Count:  5


Suggestions
The text Classification Report

class2    22
class1     6
class3     5
Name: pred_sentiment, dtype: int64
The Maximum Class output is:  22


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
